In [1]:
import numpy as np
import numba as nb
import random
import matplotlib.pyplot as plt

## 4: Multiplying Matrices
- test that $\sigma_j\sigma_k = \delta_{jk}\mathbf{1} + i \epsilon_{jkl}\sigma_l$

In [2]:
#from 2024-1
@nb.jit(nopython=True)
def norm_diff(L, M):
    LminusM = L - M
    mag_LminusM = np.sqrt(np.sum(np.abs(LminusM)**2))
    mag_L = np.sqrt(np.sum(np.abs(L)**2))
    mag_M = np.sqrt(np.sum(np.abs(M)**2))
    
    delta = mag_LminusM / (mag_L + mag_M)
    
    return delta

## 5: Create function to multiply matrices
- (5a): MAKE GENERAL MATRIX MAKER: create function that inputs 2 arrays of complex numbers ($A_0, \vec{A}$) and ($B_0, \vec{B}$) and output array of complex numbers: ($C_0, \vec{C}$)
- (5b) create function that outputs array of 4 random complex numbers
- (5c) Test function using random A and B arrays compare two ways

In [3]:
@nb.jit(nopython=True)
def inner_product(v,w):
    vx, vy, vz = v
    wx, wy, wz = w
    dot = vx*wx + vy*wy +vz*wz
    return dot

@nb.jit(nopython=True)
def inner_product1(v,w):
    return v[0]*w[0] + v[1]*w[1] +v[2]*w[2]

In [4]:
v = np.array([1,1,1])
w = np.array([1,2,2])
p = inner_product1(v,w)
p

5

In [4]:
##5a: make general matrix maker
@nb.jit(nopython=True)
def matrix_maker(m0xyz): #array to matrix
    M0, Mx, My, Mz = m0xyz
    
    identity_matrix = np.zeros((2,2), dtype = 'complex128')
    identity_matrix[0,0]= 1
    identity_matrix[1,1] = 1

    pauli_x = np.zeros((2,2), dtype = 'complex128')
    pauli_x[0,1]= 1
    pauli_x[1,0] = 1

    pauli_y = np.zeros((2,2), dtype = 'complex128')
    pauli_y[0,1]= -1j
    pauli_y[1,0] = 1j

    pauli_z = np.zeros((2,2), dtype = 'complex128')
    pauli_z[0,0]= 1
    pauli_z[1,1] = -1
    
    m = np.array([Mx, My, Mz])
    
    pauli_basis = np.array([pauli_x, pauli_y, pauli_z])
    
    m_dot_pauli = Mx*pauli_x + My*pauli_y + Mz*pauli_z
    
    matrix = M0 * identity_matrix + m_dot_pauli
    
    return matrix

In [5]:
a0xyz = np.array([1, 1, 1, 1])
b0xyz = np.array([2, 2, 2, 2])
A = matrix_maker(a0xyz)
B = matrix_maker(b0xyz)
C = A@B
print(A)
print(B)
print(C)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mNo implementation of function Function(<built-in function array>) found for signature:
 
 >>> array(list(array(complex128, 2d, C))<iv=None>)
 
There are 4 candidate implementations:
[1m  - Of which 4 did not match due to:
  Overload in function '_OverloadWrapper._build.<locals>.ol_generated': File: numba/core/overload_glue.py: Line 131.
    With argument(s): '(list(array(complex128, 2d, C))<iv=None>)':[0m
[1m   Rejected as the implementation raised a specific error:
     TypingError: Failed in nopython mode pipeline (step: nopython frontend)
   [1m[1m[1mNo implementation of function Function(<intrinsic stub>) found for signature:
    
    >>> stub(list(array(complex128, 2d, C))<iv=None>)
    
   There are 2 candidate implementations:
   [1m  - Of which 2 did not match due to:
     Intrinsic in function 'stub': File: numba/core/overload_glue.py: Line 35.
       With argument(s): '(list(array(compl

In [ ]:
a0xyz = np.array([1, 1, 1, 1])
b0xyz = np.array([2, 2, 2, 2])
A = matrix_maker(a0xyz)
B = matrix_maker(b0xyz)
C = A@B
print(A)
print(B)
print(C)

In [ ]:
## takes two arrays and creates a third that can then be used to make a matrix
def calculate_c0xyz(a0xyz, b0xyz): #arrays to array
    A0, Ax, Ay, Az = a0xyz
    B0, Bx, By, Bz = b0xyz
    axyz = np.array([Ax, Ay, Az])
    bxyz = np.array([Bx, By, Bz])
    C0 = A0*B0 + np.tensordot(axyz, bxyz, axes=1)
    cxyz = A0*bxyz + B0*axyz +1j*(np.cross(axyz, bxyz))
    c0xyz = ([C0, cxyz[0], cxyz[1], cxyz[2]])
    return c0xyz

In [ ]:
#testing calculate_c0xyz
e0xyz = [1, 1, 1, 1]
f0xyz = [2, 2, 2, 2]
g0xyz = calculate_c0xyz(e0xyz, f0xyz)
print(g0xyz)
G = matrix_maker(g0xyz)
print(G)

In [ ]:
##5b make random number generator
    ## np.random.randn(n) returns a 1D array with n random numbers, so here it makes a 1D array of 4 random numbers (could make a matrix of random numbers or a 2D array i should say, if you give two inputs like np.random.randn(m,n))
    ## and "randn" refers to "random normal" which means it generates numbers from the standard normal distribution (gaussian distribution)
def random_complex_numbers():
    real_parts = np.random.randn(4)
    imaginary_parts = np.random.randn(4)
    complex_numbers = real_parts + 1j*imaginary_parts
    return complex_numbers

random_numbers = random_complex_numbers()
print(random_numbers)

In [ ]:
#take a0xyz and b0xyz and make c0xyz then C
def one_matrix_test(a0xyz, b0xyz): #arrays to matrix
    c0xyz = calculate_c0xyz(a0xyz, b0xyz)
    C = matrix_maker(c0xyz)
    return C

#take a0xyz and b0xyz and make A and B then C
def three_matrix_test(a0xyz, b0xyz): #arrays to matrix
    A = matrix_maker(a0xyz)
    B = matrix_maker(b0xyz)
    C = A@B
    return C

In [ ]:
#testing
h0xyz = [1, 1, 1, 1]
i0xyz = [2, 2, 2, 2]
C1 = one_matrix_test(h0xyz, i0xyz)
C2 = three_matrix_test(h0xyz, i0xyz)

In [ ]:
norm_diff(C1, C2)
print(C1)
print (C2)

In [ ]:
### 5c: testing function using random numbers
def trial_loop(N): 

    delta_values = [] #an empty list so i can append later
    random_numbers_used = []

    for i in range(N):
        #create random array
        random_numbers1 = random_complex_numbers()
        random_numbers2 = random_complex_numbers()

        #compute density matrix both ways
        L = one_matrix_test(random_numbers1, random_numbers2)
        M = three_matrix_test(random_numbers1, random_numbers2)

        #compare the matrices
        delta = norm_diff(L, M)

        #save the delta values into an array and append the random arrays to a list
        delta_values.append(delta)
        random_numbers_used.append((random_numbers1, random_numbers2)) #called it something different because i was trying to append random numbers to itslef and it said no
    
    return delta_values, random_numbers_used

In [ ]:
N = 100
delta_values, random_numbers_used = trial_loop(N) #run like this to capture outputs

print("DELTA VALUES:", delta_values)
print("RANDOM NUMBERS USED:", random_numbers_used)

## 6: Create matrix multiplication for $(\mathbf{1} - \rho^{(2)})\rho^{(4)}$
- (6a): create functions that input $(P_0, \vec{P})$ and output $(A_0, \vec{A})$ and $(B_0, \vec{B})$
- (6b): test

In [ ]:
## 6a
def array_for_rho4(p0xyz): 
    P0, Px, Py, Pz = p0xyz
    pxyz = np.array([Px, Py, Pz])
    B0 = 0.5*P0
    bxyz = 0.5*P0*pxyz
    b0xyz = ([B0, bxyz[0], bxyz[1], bxyz[2]])
    return b0xyz #makes array to be put into matrix maker to create rho

def array_for_rho2(p0xyz):
    P0, Px, Py, Pz = p0xyz
    pxyz = np.array([Px, Py, Pz])
    A0 = 1 - 0.5*P0
    axyz = -0.5*P0*pxyz
    a0xyz = ([A0, axyz[0], axyz[1], axyz[2]])
    return a0xyz
#i think the point of this is to take the specific array and matrix and make it general so I can put into general matrix maker. If i was using create_density_matrix, i could probably put p0xyz directly in and it would make the matrix?

^ i think the point of this is to take the specific array and matrix and make it general so I can put into general matrix maker. If i was using create_density_matrix, i could probably put p0xyz directly in and it would make the matrix?

In [ ]:
## 6b testing
p0xyz = [1, 1, 1, 1]
b0xyz = array_for_rho4(p0xyz) #how to check this
print('ARRAY B:', b0xyz)
rho4 = matrix_maker(b0xyz)
print('RHO4:',rho4)

p0xyz = [1, 1, 1, 1]
a0xyz = array_for_rho2(p0xyz) #how to check this
print('ARRAY A:', a0xyz)
rho2 = matrix_maker(a0xyz)
print('RHO2:', rho2)

## 7: Calculate $ C = (\mathbf{1} - \rho^{(2)})\rho^{(4)} = C_0\mathbf{1} + \vec{C}* \vec{\sigma}$
- (7a): create function that inputs 2 arrays ($P_0^{(2)}, \vec{P^{(2)}}$) and ($P_0^{(4)}, \vec{P^{(4)}}$) and outputs $(C_0, \vec{C})$
- (7b): test

- $\textbf{create_rho2}$ creates an array that can be put into matrix maker to create matrix "rho2"
- $\textbf{create_rho4}$ creates an array that can be put into matrix maker to create matrix "rho4"

- "rho2" is a matrix:  $(\mathbf{1} - \rho^{(2)})$
- "rho4" is a matrix: $\rho^{(4)}$

- so to create rho2 or rho4, need to take p2 or p4 and input it into the correct function, then we will have an array to put into matrix maker to get rho2 and rho4?
..... i am getting lost

In [ ]:
#### dont need this: do need this!!!
def rho2_matrix(p0xyz_2):
    rho2_array = array_for_rho2(p0xyz_2)
    rho2 = matrix_maker(rho2_array)
    return rho2

def rho4_matrix(p0xyz_4):
    rho4_array = array_for_rho4(p0xyz_4)
    rho4 = matrix_maker(rho4_array)
    return rho4

In [ ]:
##7a
def rho2timesrho4(p0xyz_2, p0xyz_4):
    #make a0xyz and b0xyz from p0xyz_2 and p0xyz_4 -- thats what array_for_rho4 functions do!
    a0xyz = array_for_rho2(p0xyz_2)
    b0xyz = array_for_rho4(p0xyz_4)
    c0xyz = calculate_c0xyz(a0xyz, b0xyz)
    return c0xyz

p0xyz_2 = [1, 1, 1, 1]
p0xyz_4 = [1, 1, 1, 1]
c10xyz = rho2timesrho4(p0xyz_2, p0xyz_4)
C1 = matrix_maker(c10xyz)
print(c10xyz)
print(C1)

COMPARING FUNCTIONS:
- function 1, $\textbf{final_C_1}$ is taking in p2 and p4 arrays, creating a0xyz and b0xyz arrays to then get a third array c0xyz which can go into matrix maker and get the final matrix C
- function 2, $\textbf{final_C_2}$ is taking in p2 and p4 arrays and making rho2 and rho4 matrices to then be multiplied and create matrix C


In [ ]:
##7b: comparing functions:
%time
def final_C_1(p0xyz_2, p0xyz_4):
    #make a0xyz and b0xyz from p0xyz_2 and p0xyz_4 -- thats what array_for_rho4 functions do!
    a0xyz = array_for_rho2(p0xyz_2)
    b0xyz = array_for_rho4(p0xyz_4)
    c0xyz = calculate_c0xyz(a0xyz, b0xyz)
    C1 = matrix_maker(c0xyz)
    return C1
%time
def final_C_2(p0xyz_2, p0xyz_4):
    rho2 = rho2_matrix(p0xyz_2)
    rho4 = rho4_matrix(p0xyz_4)
    C2 = rho2@rho4
    return C2

In [ ]:
p0xyz_2 = [1, 1, 1, 1]
p0xyz_4 = [1, 1, 1, 1]
C1 = final_C_1(p0xyz_2, p0xyz_4)
C2 = final_C_2(p0xyz_2, p0xyz_4)
print (C1)
print (C2)

In [ ]:
### testing with random numbers
def trial_loop(N):

    delta_values = [] #an empty list so i can append later
    random_numbers_used = []

    for i in range(N):
        #create random array
        random_numbers1 = random_complex_numbers()
        random_numbers2 = random_complex_numbers()
        
        L = final_C_1(random_numbers1, random_numbers2)
        M = final_C_2(random_numbers1, random_numbers2)

        #compare the matrices
        delta = norm_diff(L, M)

        #save the delta values into an array and append the random arrays to a list
        delta_values.append(delta)
        random_numbers_used.append((random_numbers1, random_numbers2)) #called it something different because i was trying to append random numbers to itslef and it said no
    
    return delta_values, random_numbers_used

In [ ]:
N = 100 
delta_values, random_numbers_used = trial_loop(N)

print("DELTA VALUES:", delta_values)
print("RANDOM NUMBERS USED:", random_numbers_used)

## 8: Traces
- (8a): find $D = C + \mathrm{trc}(C)\mathbf{1} = D_0\mathbf{1} + \vec{D} * \vec{\sigma}$
- (8b): Test the equivalent of $(D_0, \vec{D})$ and $D = C + \mathrm{trc}(C)\mathbf{1}$

In [ ]:
##8a
c0xyz = rho2timesrho4(p0xyz_2, p0xyz_4) #arrays were generated in cell 18
print('check C array:', c0xyz)

In [ ]:
C = final_C_1(p0xyz_2, p0xyz_4) #can use either function c1 or c2 both make the matrix c
print ('check matrix C:', C)

In [ ]:
trC = np.trace(C)
print('trace of C=', trC, '2C_0=', 2*c0xyz[0])

identity_matrix = np.zeros((2,2), dtype = 'complex128')
identity_matrix[0,0]= 1
identity_matrix[1,1] = 1

In [ ]:
D = C + np.trace(C)*identity_matrix
print('check matrix D:', D)

In [ ]:
#making matrix D another way: takes matrix D and produces d0xyz?
D2 = C + 2*c0xyz[0]
print (D)
print (D2)

## 9: Creating the real functions!
- create functions to calculate E and F:
- $E = \rho^{(3)}\big[(\mathbf{1} -\rho^{(2)})\rho^{(4)} + \mathrm{tr}\big((\mathbf{1} -\rho^{(2)})\rho^{(4)}\big)\big] $
    - $E = \rho^{(3)} \big[C + \mathrm{tr}(C)\big] $
- $F = (\mathbf{1} - \rho^{(1)}) \rho^{(3)}\big[(\mathbf{1} -\rho^{(2)})\rho^{(4)} + \mathrm{tr}\big((\mathbf{1} -\rho^{(2)})\rho^{(4)}\big)\big] $
    - $F = (\mathbf{1} - \rho^{(1)}) \rho^{(3)}\big[C + \mathrm{tr}(C)\big] $

#### THE BIG IDEA:

so the idea will be to slowly build up to function F
- first we create $(\mathbf{1} -\rho^{(2)})\rho^{(4)}$, which we call C
    - call $\mathbf{rho2timesrho4}$(p0xyz_2, p0xyz_4) and it returns c0xyz
    - the you would do matrix C = $\textbf{matrix_maker}$(c0xyz)
    - OR call $\textbf{final_C_1}$(p0xyz_2, p0xyz_4) 
    - ***** use final c1 or final c2?

- then we will create $(\mathbf{1} -\rho^{(2)})\rho^{(4)}+ \mathrm{tr}\big((\mathbf{1} -\rho^{(2)})\rho^{(4)}\big)$, which we will call D (just adding trace * identity onto C)
- then we will multiply matrix $\rho^{(3)}$, which we will call E
- lastly, we will multiply matrix $(\mathbf{1} -\rho^{(1)})$, which is then F

- Input: $P^{(1)}$, $P^{(2)}$, $P^{(3)}$, $P^{(4)}$, which I call p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4
- (10a): Output: $(2\mathrm{Re}[F_0], 2\mathrm{Re}[\vec{F}])$

In [ ]:
# define final_C_1 if i change the function name...
# define identity_matrix
# need matrix_maker
# need to define P vectors
p0xyz_1 = [1, 1, 1, 1]
p0xyz_2 = [1, 1, 1, 1]
p0xyz_3 = [1, 1, 1, 1]
p0xyz_4 = [1, 1, 1, 1]

In [ ]:
## need new name for this
def matrix_mult_arrays(m10xyz, m20xyz): #arrays to array
    M10, M1x, M1y, M1z = m10xyz
    M20, M2x, M2y, M2z = m20xyz
    m1xyz = np.array([M1x, M1y, M1z])
    m2xyz = np.array([M2x, M2y, M2z])
    M30 = M10*M20 + np.tensordot(m1xyz, m2xyz, axes=1)
    m3xyz = M10*m2xyz + M20*m1xyz +1j*(np.cross(m1xyz, m2xyz))
    m30xyz = ([M30, m3xyz[0], m3xyz[1], m3xyz[2]])
    return m30xyz

In [ ]:
c0xyz =[2, 1, 1, 1]
d0xyz = [c0xyz[0] + 2*c0xyz[0], c0xyz[1], c0xyz[2], c0xyz[3]]
print (d0xyz)

In [ ]:
## okay so we are going to make 2 functions to calculate F.... 
## one does the RHS which does matrix all the way
## the other does the LHS which keeps arrays all the way

#this function is for testing
def F_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    C = final_C_1(p0xyz_2, p0xyz_4)
    #print ('C=', C)
    D = C + np.trace(C)*identity_matrix
    #print ('D=', D)
    E = matrix_maker(p0xyz_3)@D
    #print ('E=', E)
    F = matrix_maker(p0xyz_1)@E
    return (F)

#this function will go into code
def F_array_to_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    #convert p's to a and b - what function does this? array_for_rho2 and array_for_rho4
    a0xyz = array_for_rho2(p0xyz_2)
    b0xyz = array_for_rho4(p0xyz_4)
   
    #multiply a and b
    c0xyz = matrix_mult_arrays(a0xyz, b0xyz)

    #add d
    d0xyz = [c0xyz[0] + 2*c0xyz[0], c0xyz[1], c0xyz[2], c0xyz[3]]
   
    #multiply p0xyz_3 to get e
    e0xyz = matrix_mult_arrays(p0xyz_3, d0xyz)
   
    #multiply p0xyz_1 to get f
    f0xyz = matrix_mult_arrays(p0xyz_1, e0xyz)
   
    #make array matrix
    F = matrix_maker(f0xyz)
    return (F)

In [ ]:
p0xyz_1 = [1, 1, 1, 1]
p0xyz_2 = [1, 1, 1, 1]
p0xyz_3 = [1, 1, 1, 1]
p0xyz_4 = [1, 1, 1, 1]

print ('INPUT P ARRAYS')
print('P1=', p0xyz_1)
print('P2=', p0xyz_2)
print('P3=', p0xyz_3)
print('P4=', p0xyz_4)

print ('CONVERTED ARRAYS')
print('a0xyz=', array_for_rho2(p0xyz_2))
print('b0xyz=', array_for_rho4(p0xyz_4))
print('c0xyz=', matrix_mult_arrays(a0xyz, b0xyz))

print ('MATRICES')
print ('A=',rho2_matrix(p0xyz_2))
print ('B=', rho4_matrix(p0xyz_4))

print ('F made with matrices', F_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4))

print ('F made with arrays', F_array_to_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4))

# compare both ways
FM = F_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
FA = F_array_to_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
print(type(FA))
print(type(FM))

In [ ]:
def trial_loop(N):

    delta_values = [] #an empty list so i can append later
    random_numbers_used = []

    for i in range(N):
        #create random array
        random_numbers1 = random_complex_numbers()
        random_numbers2 = random_complex_numbers()
        random_numbers3 = random_complex_numbers()
        random_numbers4 = random_complex_numbers()

        #compute density matrix both ways
        L = F_matrix(random_numbers1, random_numbers2, random_numbers3, random_numbers4)
        M = F_array_to_matrix(random_numbers1, random_numbers2, random_numbers3, random_numbers4)

        #compare the matrices
        delta = norm_diff(L, M)

        #save the delta values into an array and append the random arrays to a list
        delta_values.append(delta)
        random_numbers_used.append((random_numbers1, random_numbers2, random_numbers3, random_numbers4)) #called it something different because i was trying to append random numbers to itslef and it said no
   
    return delta_values, random_numbers_used

In [ ]:
N = 100
delta_values, random_numbers_used = trial_loop(N)

print("DELTA VALUES:", delta_values)
print("RANDOM NUMBERS USED:", random_numbers_used)

## 10: Function from Bennett et al
- make function that inputs the 4 p arrays and outputs an array of REAL NUMBERS so output: $(2\mathrm{Re}[F_0], 2\mathrm{Re}[\vec{F}])$

testing complex conjugate stuff:

In [ ]:
# found np.conjugate(): function that takes the complex conjugate
z = 3 + 2*1j
z_conj = np.conjugate(z)
print(z, z_conj)

In [ ]:
z = [1, -1j, 1j, 1]
z_conj = np.conjugate(z)
print(z, z_conj)

In [ ]:
def F_array(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4): #same function as before but minus making the matrix at the end
    a0xyz = array_for_rho2(p0xyz_2) # (1-rho) 
    b0xyz = array_for_rho4(p0xyz_4)
    c0xyz = matrix_mult_arrays(a0xyz, b0xyz)
    d0xyz = np.array([c0xyz[0] + 2*c0xyz[0], c0xyz[1], c0xyz[2], c0xyz[3]])
    e0xyz = matrix_mult_arrays(p0xyz_3, d0xyz)
    f0xyz = matrix_mult_arrays(p0xyz_1, e0xyz)
    return (f0xyz)

def F_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    C = final_C_1(p0xyz_2, p0xyz_4)
    D = C + np.trace(C)*identity_matrix
    E = matrix_maker(p0xyz_3)@D
    F = matrix_maker(p0xyz_1)@E
    return (F)

p0xyz_1 = [1j, 1, 1, 1]
p0xyz_2 = [1, 1, 1, 1]
p0xyz_3 = [1, 1, 1, 1]
p0xyz_4 = [1, 1, 1, -1j]

f0xyz = F_array(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
f0xyz_conj = np.conjugate(f0xyz)

print('ARRAY STUFF')
print('f0xyz=',f0xyz)
print('f0xyz_conj=',f0xyz_conj)
Re_f0xyz = f0xyz + f0xyz_conj
print('Re_f0xyz', Re_f0xyz)

Re_F_made_w_arrays = matrix_maker(Re_f0xyz)
print(Re_F_made_w_arrays)

F = F_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
F_conj = np.conjugate(F)

print('MATRIX STUFF')
print('F=',F)
print('F_conj=',F_conj)
Re_F = F + F_conj
print('Re_F=',Re_F)

In [ ]:
## 10a:
# the funciton as is outputs the F matrix, we want to take that and
# take the real part and print out the array of that
def Fvvsc_array(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    f0xyz = F_array(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    f0xyz_conj = np.conjugate(f0xyz)
    Re_f0xyz = f0xyz + f0xyz_conj
    return Re_f0xyz

def Fvvsc_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    F = F_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    F_conj = np.conjugate(F)
    F_dagger = np.transpose(F_conj)
    Fvvsc = F + F_dagger
    return Fvvsc

def arr_2_mat(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    Re_f0xyz = Fvvsc_array(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    Fvvsc_from_array = matrix_maker(Re_f0xyz)
    return Fvvsc_from_array

In [ ]:
p0xyz_1 = [1j, 1, 1, 1]
p0xyz_2 = [1, 1, 1, 1]
p0xyz_3 = [1, 1, 1, 1]
p0xyz_4 = [1, 1, 1, -1j]


Re_f0xyz = Fvvsc_array(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
Fvvsc_from_array = matrix_maker(Re_f0xyz)
print('Fvvsc_array PRODUCES:')
print(Fvvsc_from_array)

Fvvsc_arr_2_mat = arr_2_mat(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
print('arr_2_mat PRODUCES:')
print(Fvvsc_arr_2_mat)

F_made_with_matrices = Fvvsc_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
print('Fvvsc_matrix PRODUCES:')
print(F_made_with_matrices)

In [ ]:
## testing
def trial_loop(N):

    delta_values = [] #an empty list so i can append later
    random_numbers_used = []

    for i in range(N):
        #create random array
        random_numbers1 = random_complex_numbers()
        random_numbers2 = random_complex_numbers()
        random_numbers3 = random_complex_numbers()
        random_numbers4 = random_complex_numbers()

        #compute density matrix both ways   
        L = arr_2_mat(random_numbers1, random_numbers2, random_numbers3, random_numbers4)
        M = Fvvsc_matrix(random_numbers1, random_numbers2, random_numbers3, random_numbers4)

        #compare the matrices
        delta = norm_diff(L, M)

        #save the delta values into an array and append the random arrays to a list
        delta_values.append(delta)
        random_numbers_used.append((random_numbers1, random_numbers2, random_numbers3, random_numbers4)) #called it something different because i was trying to append random numbers to itslef and it said no
   
    return delta_values, random_numbers_used

In [ ]:
N = 2
delta_values, random_numbers_used = trial_loop(N)

print("DELTA VALUES:", delta_values)
print("RANDOM NUMBERS USED:", random_numbers_used)

In [ ]:

#two ways to make final F
def F_array(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4): #same function as before but minus making the matrix at the end
    a0xyz = p0xyz2_to_a0xyz(p0xyz_2)
    b0xyz = p0xyz4_to_b0xyz(p0xyz_4)
    c0xyz = matrix_mult_arrays(a0xyz, b0xyz)
    d0xyz = [c0xyz[0] + 2*c0xyz[0], c0xyz[1], c0xyz[2], c0xyz[3]]
    e0xyz = matrix_mult_arrays(p0xyz_3, d0xyz)
    f0xyz = matrix_mult_arrays(p0xyz_1, e0xyz)
    return (f0xyz)
#note: can probably refine F_array_to_matrix function to just call F_array and then matrix maker
def F_array_to_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    a0xyz = p0xyz2_to_a0xyz(p0xyz_2)
    b0xyz = p0xyz4_to_b0xyz(p0xyz_4)
    c0xyz = matrix_mult_arrays(a0xyz, b0xyz)
    d0xyz = [c0xyz[0] + 2*c0xyz[0], c0xyz[1], c0xyz[2], c0xyz[3]]
    e0xyz = matrix_mult_arrays(p0xyz_3, d0xyz)
    f0xyz = matrix_mult_arrays(p0xyz_1, e0xyz)
    F = matrix_maker(f0xyz)
    return (F)

def F_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    C = p0xyz2_p0xyz4_to_C(p0xyz_2, p0xyz_4)
    D = C + np.trace(C)*identity_matrix
    E = matrix_maker(p0xyz_3)@D
    F = matrix_maker(p0xyz_1)@E
    return (F)

# RENAMING THINGS-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
#matrix 1 array, matrix 2 array --> output matrix 3 array

######### KEY:
######### takes two, 4 component vectors each corresponding to a matrix
######### multiplies the "matrices" but keeps them as vector/array representations the whole time
######### outputs the components for the resulting matrix
def multiply_matrices(M1components, M2components): 
    M10, M1x, M1y, M1z = M1components
    M20, M2x, M2y, M2z = M2components
    m1xyz = np.array([M1x, M1y, M1z])
    m2xyz = np.array([M2x, M2y, M2z])
    M30 = M10*M20 + np.tensordot(m1xyz, m2xyz, axes=1)
    m3xyz = M10*m2xyz + M20*m1xyz +1j*(np.cross(m1xyz, m2xyz))
    M3components = np.array([M30, m3xyz[0], m3xyz[1], m3xyz[2]])
    return M3components

#functions to take p arrays and make general a and b, so that we can do multiplication noramllly

######### KEY:
######### this function generalizes the specific array that is (identity minus rho)...
######### it takes the 4 component vector/array that would create a matrix that is in the form (1-rho)
######### makes it into more generalized components "a0xyz"
def generalize_IDrho(IDrho_array):
    P0, Px, Py, Pz = IDrho_array
    pxyz = np.array([Px, Py, Pz])
    A0 = 1 - 0.5*P0
    axyz = -0.5*P0*pxyz
    a0xyz = np.array([A0, axyz[0], axyz[1], axyz[2]])
    return a0xyz #makes array to be put into matrix maker to create (ID-rho)

######### KEY:
######### this function generalizes the specific array that is just rho...
######### it takes the 4 component vector/array that would create a matrix that is in the form rho
######### makes it into more generalized components "b0xyz"
def generalize_rho(rho_array):
    P0, Px, Py, Pz = rho_array
    pxyz = np.array([Px, Py, Pz])
    B0 = 0.5*P0
    bxyz = 0.5*P0*pxyz
    b0xyz = np.array([B0, bxyz[0], bxyz[1], bxyz[2]])
    return b0xyz #makes array to be put into matrix maker to create rho

######### KEY:
######### this function generalizes then multiplies:
######### first input is rho matrix that is identity minus rho (1-rho)
######### second input is rho matrix that is solo (rho)
######### takes generalized versions and multiplies them in the order (1-rho)rho
def generalize_then_mult(IDrho_array, rho_array):
    a0xyz = generalize_IDrho(IDrho_array)
    b0xyz = generalize_rho(rho_array)
    c0xyz = multiply_matrices(a0xyz, b0xyz)
    C = matrix_maker(c0xyz)
    return C

######### KEY:
######### this function generalizes then multiplies:
######### first input is rho matrix that is identity minus rho (1-rho)
######### second input is rho matrix that is solo (rho)
######### NOTE THE DIFFERENCE:
######### this function takes generalized versions and multiplies them in the order rho(1-rho)
def generalize_then_mult_term2(IDrho_array, rho_array):
    a0xyz = generalize_IDrho(IDrho_array)
    b0xyz = generalize_rho(rho_array)
    c0xyz = multiply_matrices(b0xyz, a0xyz)
    C = matrix_maker(c0xyz)
    return C

In [ ]:
#two ways to make final F (term 1) 
#NOTE: these are all not the final Fvvsc, because they have not yet added the harmonic conjugate

######### KEY:
######### this function takes four vectors: each a 4 component array to represent a matrix rho
######### it calculates the first term in Bennett et al. A.11: (1-rho1)rho3[(1-rho2)rho4 + tr(-)]
######### it outputs the resulting matrix for this term, but as its ARRAY/VECTOR representation
def F_components_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    # (1-rho2)
    a0xyz = generalize_IDrho(p0xyz_2)
    
    # rho4
    b0xyz = generalize_rho(p0xyz_4)
    
    # (1-rho2)rho4 ....aka....
    # a0xyz * b0xyz
    c0xyz = multiply_matrices(a0xyz, b0xyz)
    
    # (1-rho2)rho4 + tr(-)
    d0xyz = np.array([c0xyz[0] + 2*c0xyz[0], c0xyz[1], c0xyz[2], c0xyz[3]])
    
#now need to generalize rho3 and rho1 arrays
    
    # (1-rho1)
    g0xyz = generalize_IDrho(p0xyz_1)
    
    # rho3
    h0xyz = generalize_rho(p0xyz_3)
    
    # e0xyz = rho3[(1-rho2)rho4 + tr(-)] .... aka.....
    # e0xyz = h0xyz * d0xyz
    e0xyz = multiply_matrices(h0xyz, d0xyz)
    
    # f0xyz = (1-rho1)rho3[(1-rho2)rho4 + tr(-)] .... aka.....
    # f0xyz = g0xyz * h0xyz * d0xyz .... aka.....
    # f0xyz = g0xyz * e0xyz
    f0xyz = multiply_matrices(g0xyz, e0xyz)
    return (f0xyz)

######### KEY:
######### this function takes four vectors: each a 4 component array to represent a matrix rho
######### it calculates the first term in Bennett et al. A.11: (1-rho1)rho3[(1-rho2)rho4 + tr(-)]
######### it does this by using the components created from "F_components_term1" function then putting it into matrix_maker
def F_array_to_matrix_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    f0xyz = F_components_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    F = matrix_maker(f0xyz)
    return (F)

######### KEY:
######### this function takes four vectors: each a 4 component array to represent a matrix rho
######### it calculates the first term in Bennett et al. A.11: (1-rho1)rho3[(1-rho2)rho4 + tr(-)]
######### it outputs the resulting MATRIX for this term
def F_matrix_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    # C = (1-rho2)rho4 ..... or.....
    # C = A*B 
    C = generalize_then_mult(p0xyz_2, p0xyz_4)
    
    # D = (1-rho2)rho4 + tr((1-rho2)rho4)
    # D = C + tr(C)*I
    D = C + np.trace(C)*identity_matrix
    
    # need to generalize rho3 and rho1 arrays
    # (1-rho1) 
    g0xyz = generalize_IDrho(p0xyz_1)
    
    # rho1
    h0xyz = generalize_rho(p0xyz_3)
    
    # E = rho3[(1-rho2)rho4 + tr((1-rho2)rho4)]
    # E = rho3*[C + tr(C)*I]
    # E = rho3*D
    E = matrix_maker(h0xyz)@D
    
    # F = (1-rho1)rho3[(1-rho2)rho4 + tr((1-rho2)rho4)]
    # F = (1-rho1)rho3*[C + tr(C)*I]
    # F = (1-rho1)rho3*D
    # F = (1-rho1)*E
    F = matrix_maker(g0xyz)@E
    return (F)

## 11: Repeat but for the other term
- (11a): create another function: switch 2 and 4 and switch 1 and 3
$F = (\mathbf{1} - \rho^{(1)}) \rho^{(3)}\big[(\mathbf{1} -\rho^{(2)})\rho^{(4)} + \mathrm{tr}\big((\mathbf{1} -\rho^{(2)})\rho^{(4)}\big)\big]  - \rho^{(1)}(\mathbf{1} - \rho^{(3)})\big[\rho^{(2)}(\mathbf{1} -\rho^{(4)}) + \mathrm{tr}\big(\rho^{(2)}(\mathbf{1} -\rho^{(4)})\big)\big] + \mathrm{h.c.} $
- (11b): test it
(we will do this for all the terms in A.11)

In [ ]:
######### KEY:
######### this function takes four vectors: each a 4 component array to represent a matrix rho
######### it calculates the second term in Bennett et al. A.11: rho1(1-rho3)[rho2(1-rho4) + tr(-)]
######### it outputs the resulting matrix for this term, but as its ARRAY/VECTOR representation
def F_components_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4): #same function as before but minus making the matrix at the end
    # (1-rho4)
    a0xyz = generalize_IDrho(p0xyz_4)
    
    ######should i do it like a is always the lower rho number, or should 
    ######i do it like a is always the first matrix
    ######or like a is always the identity one (this is what I have now)--- DECIDED TO KEEP THIS WAY
    
    # rho2
    b0xyz = generalize_rho(p0xyz_2)
    
    # rho2(1-rho4)
    c0xyz = multiply_matrices(b0xyz, a0xyz)
    
    # rho2(1-rho4) + tr(-)
    d0xyz = np.array([c0xyz[0] + 2*c0xyz[0], c0xyz[1], c0xyz[2], c0xyz[3]])

    ###generalize rho3 and rho1 
    
    # (1-rho3) 
    g0xyz = generalize_IDrho(p0xyz_3)
    
    # rho1
    h0xyz = generalize_rho(p0xyz_1)

    # e0xyz = (1-rho3)[rho2(1-rho4) + tr(-)]
    # e0xyz = g0xyz * d0xyz
    e0xyz = multiply_matrices(g0xyz, d0xyz)
    
    # f0xyz = rho1(1-rho3)[rho2(1-rho4) + tr(-)]
    # f0xyz = h0xyz * g0xyz * d0xyz
    # f0xyz = h0xyz * e0xyz
    f0xyz = multiply_matrices(h0xyz, e0xyz)
    return (f0xyz)

######### KEY:
######### this function takes four vectors: each a 4 component array to represent a matrix rho
######### it calculates the second term in Bennett et al. A.11: rho1(1-rho3)[rho2(1-rho4) + tr(-)]
######### it does this by using the components created from "F_components_term2" function then putting it into matrix_maker
def F_array_to_matrix_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    f0xyz = F_components_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    F = matrix_maker(f0xyz)
    return (F)

                     
######### KEY:
######### this function takes four vectors: each a 4 component array to represent a matrix rho
######### it calculates the second term in Bennett et al. A.11: rho1(1-rho3)[rho2(1-rho4) + tr(-)]
######### it outputs the resulting MATRIX for this term
def F_matrix_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    # C = rho2(1-rho4) ..... or.....
    # C = B*A 
    C = generalize_then_mult_term2(p0xyz_4, p0xyz_2) 
    
    # D = rho2(1-rho4) + tr(rho2(1-rho4))
    # D = C + tr(C)*I
    D = C + np.trace(C)*identity_matrix
    
    # need to generalize rho3 and rho1 arrays
    # (1-rho3) 
    g0xyz = generalize_IDrho(p0xyz_3)
    
    # rho1
    h0xyz = generalize_rho(p0xyz_1)
    
    # E = (1-rho3)[rho2(1-rho4) + tr(rho2(1-rho4))]
    # E = (1-rho3)*[C + tr(C)*I]
    # E = (1-rho3)*D
    E = matrix_maker(g0xyz)@D
    
    # F = rho1(1-rho3)[rho2(1-rho4) + tr(rho2(1-rho4))]
    # F = rho1(1-rho3)*[C + tr(C)*I]
    # F = rho1(1-rho3)*D
    # F = rho1*E
    F = matrix_maker(h0xyz)@E
    return (F)

## Fvvsc with new function names

In [ ]:
#NEW FUNCTION NAMES FVVSC TERM 1
# now adding in the harmonic conjugates

######### KEY:
######### this function takes four vectors: each a 4 component array to represent a matrix rho
######### it calculates the arrays for the FIRST TERM in Bennett et al. A.11: (1-rho1)rho3[(1-rho2)rho4 + tr(-)] using F_components_term1
######### then it adds the conjugate of this array, adds it to the original array, and outputs the real array
def Fvvsc_components_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    f0xyz = F_components_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    f0xyz_conj = np.conjugate(f0xyz)
    Re_f0xyz = f0xyz + f0xyz_conj
    return Re_f0xyz

######### KEY:
######### this function takes four vectors: each a 4 component array to represent a matrix rho
######### it calculates the arrays for the FIRST TERM in Bennett et al. A.11: (1-rho1)rho3[(1-rho2)rho4 + tr(-)] using F_matrix_term1
######### then it adds the harmonic conjugate (conjugate and transpose) of this matrix, adds it to the original matrix, and outputs the hermitian matrix
def Fvvsc_matrix_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    F = F_matrix_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    F_conj = np.conjugate(F)
    F_dagger = np.transpose(F_conj)
    Fvvsc = F + F_dagger
    return Fvvsc

######## KEY:
######## takes output array from Fvvsc_components_term1 and makes matrix
def Fvvsc_arr2mat_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    Re_f0xyz = Fvvsc_components_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    Fvvsc_from_array = matrix_maker(Re_f0xyz)
    return Fvvsc_from_array

In [ ]:
#NEW FUNCTION NAMES FVVSC TERM 2

######### KEY:
######### this function takes four vectors: each a 4 component array to represent a matrix rho
######### it calculates the arrays for the SECOND TERM in Bennett et al. A.11: rho1(1-rho3)[rho2(1-rho4) + tr(-)] using F_components_term2
######### then it adds the harmonic conjugate of this array, adds it to the original array, and outputs the real array
def Fvvsc_components_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    f0xyz = F_components_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    f0xyz_conj = np.conjugate(f0xyz)
    Re_f0xyz = f0xyz + f0xyz_conj
    return Re_f0xyz

######### KEY:
######### this function takes four vectors: each a 4 component array to represent a matrix rho
######### it calculates the arrays for the SECOND TERM in Bennett et al. A.11: rho1(1-rho3)[rho2(1-rho4) + tr(-)] using F_matrix_term2
######### then it adds the harmonic conjugate (conjugate and transpose) of this matrix, adds it to the original matrix, and outputs the hermitian matrix
def Fvvsc_matrix_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    F = F_matrix_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    F_conj = np.conjugate(F)
    F_dagger = np.transpose(F_conj)
    Fvvsc = F + F_dagger
    return Fvvsc

######## KEY:
######## takes output array from Fvvsc_components_term2 and makes matrix
def Fvvsc_arr2mat_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    Re_f0xyz = Fvvsc_components_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    Fvvsc_from_array = matrix_maker(Re_f0xyz)
    return Fvvsc_from_array

## FULL Fvvsc, BOTH TERMS

In [ ]:
#now for the full term!

######## KEY:
######## this creates the full Bennet A.11 equation in ARRAY/VECTOR/COMPONENT form:
######## it does this by taking in those 4 arrays corresponding to 4 matrices
######## and then it uses Fvvsc_components_term1 and Fvvsc_components_term2 
######## to output the full array/vector representatation of the full Fvvsc term
def Fvvsc_components(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    term1 = Fvvsc_components_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    term2 = Fvvsc_components_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    return term1 + term2

######## KEY:
######## this creates the full Bennet A.11 equation in MATRIX form: 
######## it does this by taking in those 4 arrays corresponding to 4 matrices
######## and then it uses Fvvsc_matrix_term1 and Fvvsc_matrix_term2 
######## to output the full MATRIX of the full Fvvsc term
def Fvvsc_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    term1 = Fvvsc_matrix_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    term2 = Fvvsc_matrix_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    return term1 + term2

######## KEY:
######## uses Fvvsc_arr2mat_term1 and Fvvsc_arr2mat_term2 to make the full MATRIX of the full Fvvsc term
def Fvvsc_arr2mat(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    term1 = Fvvsc_arr2mat_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    term2 = Fvvsc_arr2mat_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    return term1 + term2

In [ ]:
p0xyz_1 = [1j, 1, 1, 1]
p0xyz_2 = [1, 1, 1, 1]
p0xyz_3 = [1, 1, 1, 1]
p0xyz_4 = [1, 1, 1, -1j]

def trial_loop(N):

    delta_values = [] #an empty list so i can append later
    random_numbers_used = []

    for i in range(N):
        #create random array
        random_numbers1 = random_complex_numbers()
        random_numbers2 = random_complex_numbers()
        random_numbers3 = random_complex_numbers()
        random_numbers4 = random_complex_numbers()

        #compute density matrix both ways   
        L = Fvvsc_arr2mat(random_numbers1, random_numbers2, random_numbers3, random_numbers4)
        M = Fvvsc_matrix(random_numbers1, random_numbers2, random_numbers3, random_numbers4)

        #compare the matrices
        delta = norm_diff(L, M)

        #save the delta values into an array and append the random arrays to a list
        delta_values.append(delta)
        random_numbers_used.append((random_numbers1, random_numbers2, random_numbers3, random_numbers4)) #called it something different because i was trying to append random numbers to itslef and it said no
   
    return delta_values, random_numbers_used

In [ ]:
N = 100
delta_values, random_numbers_used = trial_loop(N)

print("DELTA VALUES:", delta_values)
print("RANDOM NUMBERS USED:", random_numbers_used)